###Imports

In [2]:
from matplotlib import rc
from matplotlib import colors
from matplotlib.patches import FancyArrowPatch
rc('animation', html='jshtml')

from math import *
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import matplotlib.animation as animation
import abc
from numpy.typing import NDArray

from scipy import stats


### Game Plotter.
Plots are generated with game history and actions of agents. You should not need to edit this code.

In [3]:
#generate a fake game with walls and agents
walls = np.random.binomial(1, 0.2, (8,8))
test_map = [np.random.randint(0,4, (8,8)) for i in range(10)]
for i in test_map:
  i[i==1] = 0
  i[walls== 1] = 1
# [empty, wall, friendly, enemy]

#generate actions taken by agents
action_hist = [(np.random.randint(0, 25, (8,8)),np.random.randint(11, 13, (8,8))) for _ in range(10)]

# RGB color definitions
color_empty = (0.98, 0.98, 0.98)  # Slightly off-white for empty cells
color_wall = (0.3, 0.3, 0.3)  # Dark grey for walls
color_p1 = (0.8, 0.2, 0.2)  # Less saturated red for player 1
color_p2 = (0.2, 0.2, 0.8)  # Less saturated blue for player 2
color_p1_attack = (1.0, 0.5, 0.5)  # Light red for player 1 attack
color_p2_attack = (0.5, 0.5, 1.0)  # Light blue for player 2 attack


import matplotlib.pyplot as plt
from matplotlib.patches import Arc, RegularPolygon
import numpy as np
from numpy import radians as rad

#draw the circle
def drawCirc(ax,radius,centX,centY,angle_,theta2_, line_width = 3,color_='black'):
    #========Line
    arc = Arc([centX,centY],radius,radius,angle=angle_,
          theta1=0,theta2=theta2_,capstyle='round',linestyle='-',lw=line_width,color=color_)
    ax.add_patch(arc)


# draw the attack lines from agents
def calculate_attack_lines(frame_matrix, action_matrix, player_number):
    lines = []
    for i in range(frame_matrix.shape[0]):
        for j in range(frame_matrix.shape[1]):
            if frame_matrix[i, j] == player_number:
                action = int(action_matrix[i, j])
                direction = np.unravel_index(action, (5, 5))
                direction = (direction[0] - 2, direction[1] - 2)  # Adjust to get -2 to 2 range
                start = (j, i)  # Center of the cell

                if direction == (0, 0):  # Self-attack
                    lines.append((start, 'self'))
                else:
                    end = (start[0] + direction[1], start[1] + direction[0])  # Calculate end point
                    lines.append((start, end))

    return lines

#frame function - used to draw all elements in each frame
def frame(w, im, p1_lines, p2_lines, ax, cmap, norm):
    height, width = w.shape
    im.set_data(w)
    ax.clear()
    ax.imshow(w, cmap=cmap, norm=norm, origin = 'upper')


    # Draw attack lines for both players
    for line in p1_lines:
        if line[1] == 'self':  # Self-attack
            drawCirc(ax,0.35,line[0][0],line[0][1],90,270,line_width = 3,color_=color_p1_attack)
        else:
            ax.annotate('', xy=line[1], xytext=line[0],
                        arrowprops=dict(arrowstyle="->, head_length=.9, head_width=0.3", color=color_p1_attack, linewidth=3), annotation_clip = False)
    # Repeat for player 2
    for line in p2_lines:
        if line[1] == 'self':  # Self-attack
            drawCirc(ax,0.35,line[0][0],line[0][1],90,270,line_width = 3,color_=color_p2_attack)
        else:
            ax.annotate('', xy=line[1], xytext=line[0],
                        arrowprops=dict(arrowstyle="->, head_length=.9, head_width=0.3", color=color_p2_attack, linewidth=3), annotation_clip = False)

    # Rest of the function...

    # Add grid lines for visual separation of cells
    ax.set_xticks(np.arange(-0.5, width, 1), minor=True)
    ax.set_yticks(np.arange(-0.5, height, 1), minor=True)
    ax.grid(which="minor", color="grey", linestyle='-', linewidth=0.5)
    ax.tick_params(which="minor", size=0)

    return im

def anim_builder(frames, action_hist, fig_size = (4,4)):
    """
    Builds the animation with attack lines if provided.
    """
    fig, ax = plt.subplots(figsize=(6,6))
    cmap = colors.ListedColormap([color_empty, color_wall, color_p1, color_p2])
    bounds = [-0.5, 0.5, 1.5, 2.5, 3.5]
    norm = colors.BoundaryNorm(bounds, cmap.N)
    im = ax.imshow(frames[0], cmap=cmap, norm=norm)
    #h,w = frames[0].shape

    def frame_with_lines(frame_num):
        if frame_num+1 < len(action_hist):
          p1_lines = calculate_attack_lines(frames[frame_num], action_hist[frame_num+1][0], 2) if action_hist else []
          p2_lines = calculate_attack_lines(frames[frame_num], action_hist[frame_num+1][1], 3) if action_hist else []
        else:
          p1_lines = []
          p2_lines = []
        return frame(frames[frame_num], im, p1_lines, p2_lines, ax, cmap, norm)

    anim = animation.FuncAnimation(fig, frame_with_lines, frames=len(frames), blit=False, repeat=True)
    plt.close(fig)
    return anim

# Create the animation with attack lines
anim = anim_builder(test_map, action_hist)
#display the result in the notebook
anim

In [4]:
anim.save('test_anim.gif', writer='pillow', fps=5)

In [5]:
#call for getting adjacent cell features
# given x,y, return cells in range
def get_cell_feats(state_mat, y, x, n=1):
    rows, cols = state_mat.shape

    if not 0 <= x < cols:
        raise ValueError('Invalid X coordinate')
    if not 0 <= y < rows:
        raise ValueError('Invalid Y coordinate')

    y_indices = [(y+i) % rows for i in range(-n, n+1)]
    x_indices = [(x+i) % cols for i in range(-n, n+1)]

    return state_mat[np.ix_(y_indices, x_indices)]

##Agent Class + Example Agents. Feel Free to modify or train off of!

###The Agent Superclass.
Your code will need to extend this class.

In [6]:
class Agent():
  __metaclass__ = abc.ABCMeta

  def __init__(self,
                 obs_onehot: bool = False,
                 attack_map: bool = False,
                 attack_map_logits: bool = False,
                 ):
        """
        :param obs_onehot: Determines whether the observation that is received is one-hot encoded or not
        :type obs_onehot: bool
        :param attack_map: Determines whether you pass in a probability map for each cell's attack
        (4 dimensional array for your total action)
        :type attack_map: bool
        :param attack_map_logits: If you pass in a map for each cell's attack, this determines whether it
        is a logit or probability map
        :type attack_map_logits: bool
        """
        self.obs_onehot = obs_onehot
        self.attack_map = attack_map
        self.attack_map_logits = attack_map_logits

  @abc.abstractmethod
  def policy(self, obs: NDArray) -> NDArray:
      raise NotImplementedError("Please implement the policy method!")

  # stores game data (left to you for training)
  def cache(self, args):
    pass


###Random Agent

Choosees a random point to attack

In [7]:
class rand_agent(Agent):

  def policy(self, state_mat, action_space = None, obs_space = None):
    h = state_mat.shape[0]
    w = state_mat.shape[1]
    attack_mat =np.random.uniform(0,25,(h,w)).astype(int)

    return attack_mat.astype(int)

###First Enemy Agent

Attacks the first enemy it sees (So starting with top left going down)

In [8]:
class first_enemy_agent(Agent):
  def simple_adj(self, feat_mat):
    feat_mat = feat_mat.reshape(-1,1)

    enemy_list = np.argwhere(feat_mat >= 3)[:,0]
    empty_list = np.argwhere(feat_mat == 0)[:,0]

    if len(enemy_list)>=1: #if there is at least one enemy
      return enemy_list[0]
    elif len(empty_list)>=1: #if there is at least one empty
      return empty_list[0]
    else:
      return 12

  #get the adjacent features for a given x,y cell
  def get_cell_feats(self,state_mat, x,y, n = 2):
    h,w = state_mat.shape
    adj_width = 2*n+1

    xind,yind = np.indices((adj_width,adj_width))
    xind = (xind -n +x)%h
    yind = (yind -n +y)%w

    #gets a matrix of the adjacent cells by 1
    adj_vals = state_mat[xind,yind]
    return adj_vals

  def policy(self, state_mat, action_space = None, obs_space = None):
    attack_mat =np.zeros(state_mat.shape)

    for i in range(attack_mat.shape[0]):
      for j in range(attack_mat.shape[1]):
        attack_mat[i,j] = self.simple_adj(self.get_cell_feats(state_mat, i,j,2)) #get cell feats returns adjacent cells

    return attack_mat.astype(int)

###Random Enemy Agent

Chooses a random enemy within reach and attacks it

In [9]:
class rand_enemy_agent(Agent):

  #function for individual cell logic
  def rand_adj(self, feat_mat):

    feat_mat = feat_mat.reshape(-1,1)

    enemy_list = np.argwhere(feat_mat >=3)[:,0] #enemies are 3 or higher
    empty_list = np.argwhere(feat_mat == 0)[:,0] #empties are always 0

    if len(enemy_list)>=1:
      return np.random.choice(enemy_list,1)[0]
    if len(empty_list)>=1:
      return np.random.choice(empty_list,1)[0]
    else:
      return 12

  #Gets the values of the cells adjacent to a given x, y
  def get_cell_feats(self,state_mat, x,y, n = 2):
    h,w = state_mat.shape
    adj_width = 2*n+1

    xind,yind = np.indices((adj_width,adj_width))
    xind = (xind -n +x)%h
    yind = (yind -n +y)%w

    #gets a matrix of the adjacent cells by 1
    adj_vals = state_mat[xind,yind]
    return adj_vals

  #the global policy for this agent
  def policy(self, state_mat, action_space = None, obs_space = None):
    attack_mat =np.zeros(state_mat.shape)
    for i in range(attack_mat.shape[0]):
      for j in range(attack_mat.shape[1]):
        attack_mat[i,j] = self.rand_adj(self.get_cell_feats(state_mat, i,j, 2)) #get cell feats returns adjacent cells

    return attack_mat.astype(int)

###Defensive Random Enemy Agent


In [10]:
class def_rand_agent(Agent):

  #function for individual cell logic
  def rand_adj(self, feat_mat):

    feat_mat = feat_mat.reshape(-1,1)

    enemy_list = np.argwhere(feat_mat >=3)[:,0] #enemies are 3 or higher
    empty_list = np.argwhere(feat_mat == 0)[:,0] #empties are always 0
    friendly_list = np.argwhere(feat_mat == 2)[:,0] #friendlies are always 0

    if len(enemy_list)>=1:
      return np.random.choice(enemy_list,1)[0]
    if len(empty_list)>=1:
      if len(friendly_list) >=12:
        return np.random.choice(friendly_list,1)[0]
      return np.random.choice(empty_list,1)[0]
    else:
      return 12

  #Gets the values of the cells adjacent to a given x, y
  def get_cell_feats(self,state_mat, x,y, n = 2):
    h,w = state_mat.shape
    adj_width = 2*n+1

    xind,yind = np.indices((adj_width,adj_width))
    xind = (xind -n +x)%h
    yind = (yind -n +y)%w

    #gets a matrix of the adjacent cells by 1
    adj_vals = state_mat[xind,yind]
    return adj_vals

  #the global policy for this agent
  def policy(self, state_mat, action_space = None, obs_space = None):
    attack_mat =np.zeros(state_mat.shape)
    for i in range(attack_mat.shape[0]):
      for j in range(attack_mat.shape[1]):
        attack_mat[i,j] = self.rand_adj(self.get_cell_feats(state_mat, i,j, 2)) #get cell feats returns adjacent cells

    return attack_mat.astype(int)

###Priority Agent

This agent prioritizes low population areas when expanding.

In [11]:
from scipy import signal

class priority_agent(Agent):

  def __init__(self,
                 obs_onehot: bool = False,
                 attack_map: bool = False,
                 attack_map_logits: bool = False,
                 ):
        self.obs_onehot = obs_onehot
        self.attack_map = attack_map
        self.attack_map_logits = attack_map_logits
        self.adj_mat_ind = None
        return

  #gets the adjacent x,y coordinates to every friendly agent on the board
  def get_adj_indices(self, h, w , n = 1):
    adj_width = 2*n+1
    X,Y = np.indices((h,w))
    x,y = np.indices((adj_width, adj_width))
    x-=n
    y-=n
    X = X.reshape(h,w,1,1) + x
    Y = Y.reshape(h,w,1,1) + y

    X = X%h
    Y = Y%w

    X = X.reshape(h,w,-1)
    Y = Y.reshape(h,w,-1)
    return X, Y

  #gets the population mat from the state_mat
  def get_pop(self, state_mat):
    abs_map = (state_mat>=2) #get locations where there is an agent
    sum_mat = signal.convolve2d(abs_map, np.ones((3,3)), boundary = 'wrap', mode = 'same')
    return sum_mat

  #maximum population among all adjacent spaces to all friendly agents
  def get_max_pop(self, pop_mat):
    adj_pop = pop_mat[self.X_ind, self.Y_ind]
    max_pop = np.max(adj_pop, axis = 2)

    return max_pop


  def smart_adj(self, feat_mat, pop_mat): #helper function from individual cell perspective

    feat_mat = feat_mat.reshape(-1,1)

    enemy_list = np.argwhere(feat_mat >=3)[:,0]
    empty_list = np.argwhere((feat_mat == 0) * (pop_mat<=6))[:,0]

    if len(enemy_list)>=1: #if theres an enemy choose a random enemy
      if np.random.uniform()<0.8:
        return np.random.choice(enemy_list,1)[0]
      return 12

    elif len(empty_list)>=1: # if theres an empty list, choose the option with the lowest population
      enemy_ind = np.argmin(pop_mat[empty_list])
      return empty_list[enemy_ind]
    else:
      return 12

  def policy(self, state_mat, action_space= None, obs_space = None):
    attack_mat =np.zeros(state_mat.shape).astype(int)

    #get the indices for adjacent cells
    if self.adj_mat_ind is None:
      h,w = state_mat.shape
      self.X_ind, self.Y_ind = self.get_adj_indices(h,w,n = 2)


    X_ind = self.X_ind
    Y_ind = self.Y_ind

    pop_mat = self.get_pop(state_mat) #get the one distance population map
    max_pop = self.get_max_pop(pop_mat)

    for i in range(attack_mat.shape[0]): #for each row
      for j in range(attack_mat.shape[1]): #for each column

        current = state_mat[i,j]
        if current == 2:#only run if theres an ally cell here.

          adj_x = X_ind[i,j,:]
          adj_y = Y_ind[i,j,:]

          adj_pop = max_pop[adj_x, adj_y]
          feat_mat = state_mat[adj_x, adj_y]

          ind = self.smart_adj(feat_mat, adj_pop) #get cell feats returns adjacent cells
          attack_mat[i,j] = ind

          a_x = adj_x[ind] #x pos being attacked
          a_y = adj_y[ind] #y pos being attacked

          target = state_mat[a_x, a_y]

          if target == 0: #go back and edit the population map.
            state_mat[a_x, a_y] = 2

            adj_x = X_ind[a_x,a_y,:]
            adj_y = Y_ind[a_x,a_y,:]

            pop_mat[adj_x, adj_y]+=1
            max_pop = self.get_max_pop(pop_mat)

    return attack_mat.astype(int)



## AGENT 1
Won against random agent

In [12]:
class agent1(Agent):
    def __init__(self, obs_onehot=False, attack_map=False, attack_map_logits=False):
        super().__init__(obs_onehot, attack_map, attack_map_logits)

    def get_cell_feats(self, state_mat, x, y, n=2):
        h, w = state_mat.shape
        adj_width = 2 * n + 1
        x_indices = (np.arange(-n, n + 1) + x) % h
        y_indices = (np.arange(-n, n + 1) + y) % w
        return state_mat[np.ix_(x_indices, y_indices)]

    def decide_action(self, adjacent_cells, own_position):
        enemy_positions = np.argwhere(adjacent_cells >= 3)
        empty_positions = np.argwhere(adjacent_cells == 0)
        if enemy_positions.size > 0:
            target = np.random.choice(range(len(enemy_positions)), 1)[0]
            return enemy_positions[target][0] * 5 + enemy_positions[target][1]
        elif empty_positions.size > 0:
            min_neighbors = np.inf
            best_pos = None
            for pos in empty_positions:
                x, y = pos
                neighbors = self.get_cell_feats(adjacent_cells, x, y, 1)
                num_neighbors = np.sum(neighbors >= 2)
                if num_neighbors < min_neighbors:
                    min_neighbors = num_neighbors
                    best_pos = pos
            if best_pos is not None:
                return best_pos[0] * 5 + best_pos[1]
        return 12

    def policy(self, state_mat, action_space=None, obs_space=None):
        action_mat = np.zeros(state_mat.shape)
        for i in range(state_mat.shape[0]):
            for j in range(state_mat.shape[1]):
                if state_mat[i, j] == 2:
                    adjacent_cells = self.get_cell_feats(state_mat, i, j, 2)
                    action = self.decide_action(adjacent_cells, (i, j))
                    action_mat[i, j] = action
        return action_mat.astype(int)


## Agent 2

In [13]:
import numpy as np

class agent2:
    def __init__(self):
        pass

    def evaluate_state(self, state):
        """Evaluate the current state and return a score."""
        # Implement state evaluation logic here
        score = 0
        return score

    def get_legal_actions(self, state):
        """Generate and return legal actions from the current state."""
        # Implement logic to generate legal actions
        legal_actions = []
        return legal_actions

    def select_action(self, state):
        """Select the best action from the current state."""
        legal_actions = self.get_legal_actions(state)
        best_action = None
        best_score = float('-inf')

        for action in legal_actions:
            # Assume apply_action returns the new state after applying the action
            new_state = self.apply_action(state, action)
            score = self.evaluate_state(new_state)
            if score > best_score:
                best_score = score
                best_action = action

        return best_action

    def apply_action(self, state, action):
        """Apply an action to the state and return the new state."""
        # Implement action application logic here
        new_state = state  # Placeholder, modify as needed
        return new_state

    def play_turn(self, state):
        """Main method to be called for the agent to play its turn."""
        action = self.select_action(state)
        return self.apply_action(state, action)


## Agent 3

In [14]:
class agent3:
    def __init__(self, state_size, action_size, learning_rate=0.1, discount_rate=0.95, exploration_rate=1.0, exploration_decay=0.99, min_exploration_rate=0.01):
        # Now including state_size and action_size in the constructor
        self.state_size = state_size
        self.action_size = action_size
        self.learning_rate = learning_rate
        self.discount_rate = discount_rate
        self.exploration_rate = exploration_rate
        self.exploration_decay = exploration_decay
        self.min_exploration_rate = min_exploration_rate
        self.q_table = {}  # Use a dictionary to handle dynamic state representations

    def get_state_representation(self, state):
        # Convert state to a suitable representation
        return str(state)  # This is a placeholder; adjust based on how your states are represented

    def policy(self, state):
        """Decide on an action based on the current state using the Q-learning policy."""
        state_repr = self.get_state_representation(state)
        if state_repr not in self.q_table:
            self.q_table[state_repr] = [0] * self.action_size

        if random.uniform(0, 1) < self.exploration_rate:
            action = random.randint(0, self.action_size - 1)  # Explore
        else:
            action = np.argmax(self.q_table[state_repr])  # Exploit learned values
        return action

    # Include other methods for updating the Q-table, etc.


## Agent 4

In [15]:
class agent_4(Agent):

  def __init__(self):
    pass

  def rand_adj(self, feat_mat, pop_mat):
    feat_mat = feat_mat.reshape(-1,1)

    enemy_list = np.argwhere(feat_mat >= 3)[:,0]  # enemies are 3 or higher
    empty_list = np.argwhere(feat_mat == 0)[:,0]  # empties are always 0

    if len(enemy_list) >= 1:
      return np.random.choice(enemy_list, 1)[0]

    # Prioritize empty cells surrounded by low population
    low_pop_empty = [cell for cell in empty_list if pop_mat[cell[0], cell[1]] < 6]

    if len(low_pop_empty) >= 1:
      return np.random.choice(low_pop_empty, 1)[0]

    if len(empty_list) >= 1:
      return np.random.choice(empty_list, 1)[0]
    else:
      return None

  def get_cell_feats(self, state_mat, x, y, n=2):
    h, w = state_mat.shape
    adj_width = 2*n + 1

    xind, yind = np.indices((adj_width, adj_width))
    xind = (xind - n + x) % h
    yind = (yind - n + y) % w

    adj_vals = state_mat[xind, yind]
    return adj_vals

  def policy(self, state_mat, action_space=None, obs_space=None):
    attack_mat = np.zeros(state_mat.shape)

    pop_mat = np.zeros(state_mat.shape)
    for i in range(state_mat.shape[0]):
      for j in range(state_mat.shape[1]):
        pop_mat[i, j] = np.sum(self.get_cell_feats(state_mat, i, j, 2) >= 2)

    for i in range(state_mat.shape[0]):
      for j in range(state_mat.shape[1]):
        if state_mat[i, j] == 2:
          adj_values = self.get_cell_feats(state_mat, i, j, 2)
          attack_target = self.rand_adj(adj_values, pop_mat)
          if attack_target is not None:
            attack_mat[i, j] = attack_target

    return attack_mat.astype(int)



##Environment Code.

In [24]:
from scipy import signal


class cgol:
  def  __init__(self, agent_1, agent_2, board_h = 10, board_w = 10, attack_width = 5, max_rounds = 100,map = None):

    self.board_h = board_h
    self.board_w = board_w
    self.view_width = attack_width
    self.view_height = attack_width
    self.max_rounds = max_rounds


    if map is None:
      self.map = np.zeros((self.board_h,self.board_w))
    else:
      self.map = map

    self.agent_1 = agent_1
    self.agent_2 = agent_2

    self.over_pop = 6

    #mapping for action int to
    x_view, y_view = np.indices((5,5))
    x_view-=2
    y_view-=2
    self.x_view = x_view.flatten()
    self.y_view = y_view.flatten()

    self.reset()

  def reset(self):
    self.state_mat = np.zeros((self.board_h,self.board_w)) + self.map

    start_num = 1
    #set starting locations for agent
    game_y = np.random.randint(0,self.board_h, start_num) #starting locations for agent_1_x
    game_x = np.random.randint(0,self.board_w, start_num) #starting locations for agent_1_y
    self.state_mat[game_y, game_x] = 2

    game_y = np.random.randint(0,self.board_h, start_num) #starting locations for agent_1_x
    game_x = np.random.randint(0,self.board_w, start_num) #starting locations for agent_1_y
    self.state_mat[game_y, game_x] = 3

    #recording history
    self.state_history = []
    self.action_history = []
    self.reward_history = []
    self.count = 0

  def get_state(self):
    return self.state_mat

  #moves from cords to attack to attack_map
  #action_map - map of board with actions assigned integer based on coord
  #ex:
  # [1 3 4 5 0 9]      [sum of adjent attacks]
  # [3 4 9 8 6 2]      [                     ]
  # [1 0 6 7 5 3] ---> [                     ]
  # [0 0 0 4 5 6]      [                     ]
  def convert_action_map(self, action_matrix, mask_matrix):

    rows, cols = action_matrix.shape

    # Convert action_matrix to direction coordinates
    directions = np.array(np.unravel_index(action_matrix.ravel(), (5, 5))).T - 2
    y_coords, x_coords = np.indices(action_matrix.shape)

    # Filter directions by mask
    masked_directions = directions[mask_matrix.ravel()]
    y_attacked = (y_coords.ravel()[mask_matrix.ravel()] + masked_directions[:, 0]) % rows
    x_attacked = (x_coords.ravel()[mask_matrix.ravel()] + masked_directions[:, 1]) % cols

    # Aggregate attacks on each cell
    result = np.zeros_like(action_matrix)
    np.add.at(result, (y_attacked, x_attacked), 1)

    return result

  #action_mat_1 - the action being played for this step by agent_1
  #action_mat_2 - the action being played for this step by agent_2
  #returns - reward, done


  def get_reward(self):
    p1_reward = 0
    p2_reward = 0
    ###############CREATE YOUR OWN REWARD FUNCTION##########################
    p1_vals = 1*self.state_mat==1
    reward_kernal = np.array([[0   , .25 ,.5 , .25,  0],
                              [.25 , .5  ,.75, .5 ,.25],
                              [.5  , .75 , 1 , .75, .5],
                              [.25 , .5  ,.75, .5 ,.25],
                              [0   , .25 ,.5 , .25,  0]])
    reward = signal.convolve2d(self.state_mat, reward_kernal, boundary = 'wrap', mode = 'same')
    ########################################################################
    return (p1_reward, p2_reward)

  #allys are always 2, so modify it here.
  def modify_view(self, state_mat, agent_num = 2):
    view = np.copy(state_mat)

    view[state_mat==agent_num] = 2
    view[state_mat==2] = agent_num
    return view

  def get_agent_views(self, state_mat=None):
    state_mat = state_mat if state_mat is not None else self.state_mat
    p1_view = self.modify_view(state_mat, 2)
    p2_view = self.modify_view(state_mat, 3)
    return p1_view, p2_view

  def calculate_winner(self):
    # Flattening the 2D array and then filtering out 0 and 1
    filtered_array = self.state_mat.flatten()
    filtered_array = filtered_array[(filtered_array != 0) & (filtered_array != 1)]

    # Finding the most common value in the filtered array
    mode = stats.mode(filtered_array)
    return mode

  def step(self, actions=(None, None), is_training=True):

    #print(actions)

    p1_action, p2_action = actions
    p1_view, p2_view = self.get_agent_views()
    ##retrieve adjent coordinates
    p1_mask = p1_view==2 #where p1 sees allys
    p2_mask = p2_view==2 #where p2 sees allys

    #translate adjent actions into sum attack matrices
    p1_sum = self.convert_action_map(p1_action, p1_mask)
    p2_sum = self.convert_action_map(p2_action, p2_mask)

    #add an always occuring self attack
    p1_sum += 1*(self.state_mat==2)
    p2_sum += 1*(self.state_mat==3)


    # Calculate total attacks for each cell.
    total_attacks = p1_sum + p2_sum


    with np.errstate(divide='ignore', invalid='ignore'):
        p1_win_mask = (total_attacks > 0)&(p1_sum == total_attacks)
        p2_win_mask = (total_attacks > 0)&(p2_sum == total_attacks)

        #print(p1_win_mask)

        both_nonzero_mask = (p1_sum > 0) & (p2_sum > 0)

        random_numbers = np.random.rand(*p1_sum.shape)
        p1_prob = p1_sum / (p1_sum + p2_sum)
        p1_prob[np.isnan(p1_prob)] = 0

        p1_prob_win = random_numbers < p1_prob
        p2_prob_win = random_numbers >= p1_prob
        p1_prob_win = p1_prob_win*both_nonzero_mask
        p2_prob_win = p2_prob_win*both_nonzero_mask

        self.state_mat[p1_win_mask] = 2
        self.state_mat[p1_prob_win] = 2

        self.state_mat[p2_win_mask] = 3
        self.state_mat[p2_prob_win] = 3

    self.state_mat[self.map==1]=1

    ###ADD PUNISHMENT FOR OVERPOPULATION #################################
    abs_map = (self.state_mat>=2) #get locations where there is an agent
    sum_mat = signal.convolve2d(abs_map, np.ones((3,3)), boundary = 'wrap', mode = 'same')
    excess_mat = sum_mat>self.over_pop
    self.state_mat = self.state_mat * (1-(1*excess_mat))

    self.state_mat[self.map==1]=1 #walls always equal 1
    ###RECORD HISTORY #################################
    self.state_history.append(self.state_mat.copy())
    self.action_history.append((p1_action, p2_action))

    #####reward! modify as needed (your objective is to win)##############
    if is_training:
      reward = self.get_reward()
    else:
      reward = (0,0)

    self.reward_history.append(reward)

    #####reward! modify as needed (your objective is to win)

    #check if game end state is complete
    done = False
      #reward = 10000
    if np.max(self.state_mat) == 1:
      done = True
      #reward = -100
    if self.count >= self.max_rounds:
      done = True

    if not np.any(self.state_mat == 2):
      done = True
    if not np.any(self.state_mat == 3):
      done = True

    self.count+=1

    p1_view, p2_view = self.get_agent_views()
    return (p1_view, p2_view), reward, done

  #returns a history of the game
  def run_game(self, episodes = 10, training=True):
    episode_winner_record = []
    episodes_game_history = []
    episodes_action_history = []

    # set agent flags for training
    self.agent_1.is_training = training
    self.agent_2.is_training = training

    for episode in range(episodes):
      self.reset()
      while True:

          p1_view, p2_view = self.get_agent_views()

          # get attack actions from agent
          p1_action = self.agent_1.policy(p1_view)
          p2_action = self.agent_2.policy(p2_view)

          # Recieve state and reward from environment from chosen actions
          player_views_new, reward, done = self.step((p1_action, p2_action))
          p1_view_new, p2_view_new = player_views_new
          p1_reward, p2_reward = reward

          #print(self.action_history)
          # if training, record data
          if training:
            # example caching of data. Up to you to edit, modify, use, ect
            self.agent_1.cache((p1_view, p1_view_new, p1_action, p1_reward))
            self.agent_2.cache((p2_view, p2_view_new, p2_action, p2_reward))

          # End this episode when `done` is True
          if done:
            episode_winner_record.append(self.calculate_winner())
            episodes_game_history.append(self.state_history)
            episodes_action_history.append(self.action_history)
            break
    return episode_winner_record, episodes_game_history, episodes_action_history

##Test of the game

In [23]:
agent_1 = rand_enemy_agent()
agent_2 = agent1()


h = 15
w = 15
test_game = cgol(agent_1, agent_2,board_h = h,board_w = w, max_rounds = 50, map = np.random.binomial(1,0.1,(h,w)))

win_record, game_hist, action_hist = test_game.run_game(1, training=False)

# play the last episode/game
anim = anim_builder(game_hist[-1], action_hist[-1])

In [25]:
#plot gif style
anim

In [18]:
#save the gif for an individual game
anim.save('test_anim.gif', writer='pillow', fps=5)